In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
import datetime

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", category = matplotlib.cbook.mplDeprecation)

In [3]:
target='transaction_real_price'

In [4]:
h2o.init() # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /Users/bonnie/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/f1/2lb7b2n57j9d7ktf62rd4mkh0000gn/T/tmpc7jjgbpz
  JVM stdout: /var/folders/f1/2lb7b2n57j9d7ktf62rd4mkh0000gn/T/tmpc7jjgbpz/h2o_bonnie_started_from_python.out
  JVM stderr: /var/folders/f1/2lb7b2n57j9d7ktf62rd4mkh0000gn/T/tmpc7jjgbpz/h2o_bonnie_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,"14 days, 20 hours and 3 minutes"
H2O cluster name:,H2O_from_python_bonnie_80b2xd
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [5]:
df = h2o.import_file(path = 'data/trainPriceCleaned.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
df.describe()

Rows:4465
Cols:19




,transaction_real_price,city,exclusive_use_area,floor,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,supply_area,total_household_count_of_area_type,room_count,bathroom_count,heat_fuel_cogeneration,heat_fuel_gas,heat_type_central,heat_type_district,heat_type_individual,front_door_structure_corridor,front_door_structure_mixed,front_door_structure_stairway
type,int,int,real,int,int,int,int,real,int,int,int,int,int,int,int,int,int,int,int
mins,15150000.0,0.0,23.32,1.0,0.0,100.0,1.0,25.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,407325225.0839865,0.6326987681970885,78.98916750279955,10.478387458006718,1358.196640537514,1205.4591265397537,13.059350503919374,101.8436640537514,302.55341545352746,2.9393057110862264,1.5979843225083987,0.23829787234042554,0.7617021276595745,0.08421052631578947,0.23852183650615902,0.6772676371780515,0.26382978723404255,0.01702127659574468,0.7191489361702128
maxs,3878340000.0,1.0,244.8647,59.0,9766.0,6864.0,124.0,309.36,2960.0,6.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
sigma,316990725.1286814,0.48212352757671595,27.536207663209645,7.258355776752853,1511.328449626355,1147.2439103271977,14.478395659958464,32.50384704917918,342.17734963028045,0.6466331065494135,0.5029887953511339,0.4260899642891898,0.4260899642891898,0.27773438634871633,0.42622747160873403,0.4675736829724452,0.4407574610598276,0.12936499080697303,0.4494652253397927
zeros,0,1640,0,0,5,0,0,0,54,8,8,3401,1064,4089,3400,1441,3287,4389,1254
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,570000000.0,1.0,84.91,15.0,1391.0,1606.0,15.0,104.75,958.0,3.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1050000000.0,1.0,84.99,17.0,7876.0,5563.0,65.0,109.35,828.0,3.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,586050000.0,0.0,156.7997,13.0,857.0,390.0,4.0,198.7,154.0,3.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
X  = [name for name in df.columns if name != target]
print(X)

['city', 'exclusive_use_area', 'floor', 'total_parking_capacity_in_site', 'total_household_count_in_sites', 'apartment_building_count_in_sites', 'supply_area', 'total_household_count_of_area_type', 'room_count', 'bathroom_count', 'heat_fuel_cogeneration', 'heat_fuel_gas', 'heat_type_central', 'heat_type_district', 'heat_type_individual', 'front_door_structure_corridor', 'front_door_structure_mixed', 'front_door_structure_stairway']


In [8]:
# Set target and predictor variables
y = target
project = 'hyper'

# MODEL ONE (300)

In [9]:
runtime = 300

In [10]:
# Set up AutoML
aml = H2OAutoML(max_runtime_secs=runtime,                
                project_name = project)
                
aml.train(x=X,
          y=y,
          training_frame=df)  # Change training_frame=train

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
board = aml.leaderboard

In [12]:
board_csv = board.as_data_frame()
board_csv

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_1_AutoML_20190415_192126,25308559289889640,1.590866e+08,25308559289889640,9.537870e+07,0.331267
1,XGBoost_1_AutoML_20190415_192126,25792706929455124,1.606011e+08,25792706929455124,9.468537e+07,0.325136
2,XGBoost_grid_1_AutoML_20190415_192126_model_5,26095516169994532,1.615411e+08,26095516169994532,9.439783e+07,0.326511
3,XGBoost_2_AutoML_20190415_192126,26410633391141760,1.625135e+08,26410633391141760,9.510714e+07,0.325820
4,XRT_1_AutoML_20190415_192126,26785737850991984,1.636635e+08,26785737850991984,9.760681e+07,0.336689
5,XGBoost_grid_1_AutoML_20190415_192126_model_4,26786705406232530,1.636664e+08,26786705406232530,9.896766e+07,0.336037
6,GBM_4_AutoML_20190415_192126,27062394146374224,1.645065e+08,27062394146374224,9.787243e+07,0.331311
7,XGBoost_grid_1_AutoML_20190415_192126_model_2,27502291350696010,1.658381e+08,27502291350696010,1.010520e+08,0.340350
8,GBM_3_AutoML_20190415_192126,27587065086166110,1.660935e+08,27587065086166110,9.919472e+07,0.335290
9,DRF_1_AutoML_20190415_192126,27776273091140736,1.666622e+08,27776273091140736,9.836043e+07,0.335013


In [15]:
system_date = datetime.date.today()
print (system_date)

2019-04-15


In [16]:
board_csv['system_date'] = system_date
board_csv['runtime'] = runtime
board_csv

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle,system_date,runtime
0,GBM_1_AutoML_20190415_192126,25308559289889640,1.590866e+08,25308559289889640,9.537870e+07,0.331267,2019-04-15,300
1,XGBoost_1_AutoML_20190415_192126,25792706929455124,1.606011e+08,25792706929455124,9.468537e+07,0.325136,2019-04-15,300
2,XGBoost_grid_1_AutoML_20190415_192126_model_5,26095516169994532,1.615411e+08,26095516169994532,9.439783e+07,0.326511,2019-04-15,300
3,XGBoost_2_AutoML_20190415_192126,26410633391141760,1.625135e+08,26410633391141760,9.510714e+07,0.325820,2019-04-15,300
4,XRT_1_AutoML_20190415_192126,26785737850991984,1.636635e+08,26785737850991984,9.760681e+07,0.336689,2019-04-15,300
5,XGBoost_grid_1_AutoML_20190415_192126_model_4,26786705406232530,1.636664e+08,26786705406232530,9.896766e+07,0.336037,2019-04-15,300
6,GBM_4_AutoML_20190415_192126,27062394146374224,1.645065e+08,27062394146374224,9.787243e+07,0.331311,2019-04-15,300
7,XGBoost_grid_1_AutoML_20190415_192126_model_2,27502291350696010,1.658381e+08,27502291350696010,1.010520e+08,0.340350,2019-04-15,300
8,GBM_3_AutoML_20190415_192126,27587065086166110,1.660935e+08,27587065086166110,9.919472e+07,0.335290,2019-04-15,300
9,DRF_1_AutoML_20190415_192126,27776273091140736,1.666622e+08,27776273091140736,9.836043e+07,0.335013,2019-04-15,300


In [17]:
board_csv.to_csv('result/300/leaderboard.csv', sep='\t')

In [18]:
best_model = h2o.get_model(aml.leaderboard[2,'model_id'])
best_model

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid_1_AutoML_20190415_192126_model_5


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 4081997771173957.5
RMSE: 63890513.937312774
MAE: 42931685.867413215
RMSLE: 0.17120698343478208
Mean Residual Deviance: 4081997771173957.5

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 2.6095516169994532e+16
RMSE: 161541066.5124956
MAE: 94397834.49227324
RMSLE: 0.3265109681457912
Mean Residual Deviance: 2.6095516169994532e+16
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,94397832.0000000,3150388.5,98904528.0000000,88932304.0000000,98593896.0000000,89158424.0000000,96400016.0000000
mean_residual_deviance,26095516900000000.0000000,4161983830000000.0000000,26145776600000000.0000000,19876926100000000.0000000,33714919900000000.0000000,19228334500000000.0000000,31511625300000000.0000000
mse,26095516900000000.0000000,4161983830000000.0000000,26145776600000000.0000000,19876926100000000.0000000,33714919900000000.0000000,19228334500000000.0000000,31511625300000000.0000000
r2,0.7425417,0.0202928,0.7628383,0.7504266,0.7109876,0.7805769,0.7078791
residual_deviance,26095516900000000.0000000,4161983830000000.0000000,26145776600000000.0000000,19876926100000000.0000000,33714919900000000.0000000,19228334500000000.0000000,31511625300000000.0000000
rmse,160495952.0000000,12972401.0000000,161696560.0000000,140985552.0000000,183616224.0000000,138666272.0000000,177515136.0000000
rmsle,0.3264200,0.0054481,0.3378062,0.3182584,0.3304232,0.317402,0.3282105


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-15 19:24:17,1 min 5.767 sec,0.0,516114768.6270046,407325224.7237402,266374454394906464.0000000
,2019-04-15 19:24:17,1 min 5.834 sec,5.0,407516264.3605769,315187974.4100784,166069505718399616.0000000
,2019-04-15 19:24:17,1 min 5.902 sec,10.0,324418668.7303272,243908948.4797312,105247472620757776.0000000
,2019-04-15 19:24:17,1 min 5.980 sec,15.0,261139448.8606504,189291154.2893617,68193811751244272.0000000
,2019-04-15 19:24:17,1 min 6.079 sec,20.0,212764460.2953650,148474267.3720045,45268715564777968.0000000
,2019-04-15 19:24:17,1 min 6.205 sec,25.0,176304302.6804448,118575152.2517357,31083207143637908.0000000
,2019-04-15 19:24:17,1 min 6.328 sec,30.0,148799721.7921432,97094539.8544233,22141357205419216.0000000
,2019-04-15 19:24:17,1 min 6.453 sec,35.0,127926786.0645605,82036013.9099664,16365262592807832.0000000
,2019-04-15 19:24:18,1 min 6.599 sec,40.0,112614031.2330098,71459099.2434490,12681920030549316.0000000
,2019-04-15 19:24:18,1 min 6.747 sec,45.0,100807123.1833963,64002563.7007839,10162076084512438.0000000


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
supply_area,951330252186581991424.0000000,1.0,0.2155488
exclusive_use_area,867554643761956913152.0000000,0.9119385,0.1965672
city,786726641293445824512.0000000,0.8269753,0.1782535
total_parking_capacity_in_site,305895006818258649088.0000000,0.3215445,0.0693085
apartment_building_count_in_sites,234611064346005995520.0000000,0.2466137,0.0531573
total_household_count_in_sites,233429942567169949696.0000000,0.2453721,0.0528897
room_count,179693290880547946496.0000000,0.1888863,0.0407142
floor,178548936770544730112.0000000,0.1876834,0.0404549
total_household_count_of_area_type,159622858403682975744.0000000,0.1677891,0.0361667
heat_type_district,158832318339405053952.0000000,0.1669581,0.0359876


In [19]:
params = best_model.params
params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_grid_1_AutoML_20190415_192126_model_5',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_grid_1_AutoML_20190415_192126_model_5'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_trainPriceCleaned.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_trainPriceCleaned.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 

In [20]:
model_id = params['model_id']['actual']['name']
stopping_tolerance  = params['stopping_tolerance']['actual']
ntrees = params['ntrees']['actual']
max_depth = params['max_depth']['actual']
learn_rate = params['learn_rate']['actual']
sample_rate = params['sample_rate']['actual']
col_sample_rate = params['col_sample_rate']['actual']

In [21]:
params_json = {
    'model_id': model_id,
    'stopping_tolerance' : stopping_tolerance,
    'ntrees' : ntrees,
    'max_depth' : max_depth,
    'learn_rate' : learn_rate,
    'sample_rate' : sample_rate,
    'col_sample_rate' : col_sample_rate 
}

params_json

{'model_id': 'XGBoost_grid_1_AutoML_20190415_192126_model_5',
 'stopping_tolerance': 0.014965432360392486,
 'ntrees': 92,
 'max_depth': 10,
 'learn_rate': 0.05,
 'sample_rate': 1.0,
 'col_sample_rate': 0.8}

In [22]:
with open('daddddta.json', 'w') as f:
    json.dump(params_json, f)

In [ ]:
model_list = get_modelList(board_csv) 
a = []
for i in model_list:  
    print (i)
    model = h2o.get_model(i)
   
    
    varimp = model.varimp()
    vamp_csv = pd.DataFrame(varimp, 
                                columns = ['variable', 
                                           'relative_importance', 
                                           'scaled_importance',
                                           'percentage']) 
    vamp_csv['model_id'] = i
print ('done')
print (len(model_list))

In [ ]:
def get_all_varimp()
model_list = get_modelList(board_csv) 

tup=[]
gg = 1
for mid in model_list:  
    
    model = h2o.get_model(mid)
    varimp = model.varimp()     
    
    print("tryyyyyyyyyyyyyyyy          ", gg)
    gg+= 1
    try:
        for var_item in varimp:
            vv = []
            vv.append(mid)  
            ass = []
                       
            for tit in var_item:       
                ass.append(tit)             
                cound += 1
            item = vv + ass
            tup.append(item)         
        print('done')
    except:  
        print(mid)  
        print('pass')  
        pass
    continue

new_varimp = pd.DataFrame(tup, columns = ['model_id',
                                'variable', 
                                'relative_importance', 
                                'scaled_importance',
                                'percentage'                                
                               ]) 
            
new_varimp


